# S.VI.B: FIMS

This notebook plots and analyzes the results shown in Figure 7a -- 7f of Section VI.B: "FIMS."

To simplify reproducability of the artifact, we have created three slightly-modified copies of the FIMS software pipeline presented in:

> D. Wang, J. Zhang, J. Buhler, and J. Wang, “Real-time analysis of aerosol size distributions with the fast integrated mobility spectrometer (FIMS),” in 41st Conference of American Association for Aerosol Research (AAAR), Oct. 2023. [Online]. Available: https://aaarabstracts.com/2023/view_abstract.php?pid=752 

These include:
* `fims_execution_time_profile`: Obtains execution times with `getrusage()` for each job iteration of all tasks.
* `fims_elasticity_values_assignment`: 
* `fims_evaluate_online_adjustment`:

## Build

### Install OpenCV

***Note: The following script can be skipped on the provided virtual machine***

Refer to the OpenCV installation tutorial for more detailed instrutions:
https://docs.opencv.org/4.7.0/d7/d9f/tutorial_linux_install.html

In [ ]:
%%sh

# Download and unpack sources
wget -O opencv.zip https://github.com/opencv/opencv/archive/4.7.0.zip
wget -O opencv_contrib.zip https://github.com/opencv/opencv_contrib/archive/4.7.0.zip
unzip opencv.zip
unzip opencv_contrib.zip

# Create build directory and switch into it
mkdir -p build && cd build

# Configure
cmake -DOPENCV_EXTRA_MODULES_PATH=../opencv_contrib-4.7.0/modules ../opencv-4.7.0

# Build
make -j

# Install
sudo make install

### Compile FIMS Code

***Note: The following script can be skipped on the provided virtual machine***

In [ ]:
%%sh

# Compile Code for Profiling Execution Times
cd fims_execution_time_profile
mkdir build && cd build
cmake ..
make -j

# Compile Code for Assigning Elasticity Values
cd ../fims_elasticity_values_assignment
mkdir build && cd build
cmake ..
make -j

# Compile Code for Evaluation of Online Adjustment
cd ../fims_evaluate_online_adjustment
mkdir build && cd build
cmake ..
make -j

# Compile Code for Interference Task
cd ../overhead
make

## Profiling Execution Times

We first profile the execution times of each FIMS pipeline task. We make calls to `getrusage()` when each job completes, measuring the total CPU time (user and system) consumed by the task since the end of the prior job. This accounts for execution of the task's function plus the overhead of context switching and timer handling. To capture worst-case conditional behavior, we force recalculation of the inversion matrix with each iteration of data inversion.

The following code reproduces the results shown in Figure 7d -- 7f.

### Obtain Execution Times



In [ ]:
import subprocess
import os
import time
import sys

# Function to run the C++ program
def run_cpp_program(command, executable):
    command.append(executable)
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return result.returncode == 0  

# Function to rename the folder and create a new one
def manage_folders(old_folder, new_folder, sh, i):
    os.rename(old_folder, f"{old_folder}_{sh}_{i}")
    os.makedirs(new_folder, exist_ok=True)


run_times = 1
if len(sys.argv) > 1:
    try:
        run_times = int(sys.argv[1]) 
    except ValueError:
        print("Please provide a valid integer as the argument.")
        sys.exit(1) 

os.system("echo -1 > /proc/sys/kernel/sched_rt_runtime_us")
os.system("cpufreq-set -r -g performance")


executable = "../build/fims"  
result_folder = "run_time"
command = ["sudo"]

for i in range(0, run_times, 1):  # define how many time to run the code
    print(f'start running {i} ...')
    if run_cpp_program(command, executable):
        print(f"Run {i+1}: Success")
        # Rename the old folder and create a new one for the next run
        manage_folders(result_folder, result_folder, 'sparse', i)
    else:
        print(f"Run {i+1}: Failure")

    time.sleep(1)  # Optional: pause for 1 second between runs



